In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c dogs-vs-cats

 98% 794M/812M [00:03<00:00, 263MB/s]
100% 812M/812M [00:03<00:00, 252MB/s]


In [4]:
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [6]:
import os, shutil, pathlib

o_dir = pathlib.Path("train")
n_dir = pathlib.Path("dogs_vs_cats_small")

def make_subset(name:str, start:int, end:int):
    for category in ("cat", "dog"):
        dir = n_dir / name / category
        os.makedirs(dir)

        fnames = [f"{category}.{i}.jpg" for i in range(start, end)]

        for fname in fnames:
            shutil.copyfile(src=o_dir / fname, dst=dir / fname)

In [9]:
make_subset("train", start=0, end=1_000)
make_subset("validation", start=1_000, end=1_500)
make_subset("test", start=1_500, end=2_500)

In [10]:
from tensorflow.keras.utils import image_dataset_from_directory

train_data = image_dataset_from_directory(
    n_dir / "train",
    batch_size=32,
    image_size=(180, 180)
)

val_data = image_dataset_from_directory(
    n_dir / "validation",
    batch_size=32,
    image_size=(180, 180)
)

test_data = image_dataset_from_directory(
    n_dir / "test",
    batch_size=32,
    image_size=(180, 180)
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

augmentation = Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.2)
])

In [12]:
from tensorflow.keras.applications import vgg16

conv_base = vgg16.VGG16(
    include_top=False,
    weights="imagenet"
)

conv_base.trainable = True

58900480/58889256 [==============================] - 0s 0us/step


In [13]:
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [14]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop

inputs = Input(shape=(180, 180, 3))
x = augmentation(inputs)
x = vgg16.preprocess_input(x)
x = conv_base(x)
x = Flatten()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=RMSprop(learning_rate=1e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

MODEL_PATH = "test_model"

callbacks = [
    ModelCheckpoint(
        filepath=MODEL_PATH,
        monitor="val_loss",
        save_best_only=True
    )
]

In [16]:
model.fit(train_data,
          epochs=30,
          validation_data=val_data,
          callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 33s 280ms/step - loss: 2.9973 - accuracy: 0.7110 - val_loss: 0.5393 - val_accuracy: 0.9060
Epoch 2/30
63/63 [==============================] - 16s 248ms/step - loss: 0.9627 - accuracy: 0.8220 - val_loss: 0.2631 - val_accuracy: 0.9450
Epoch 3/30
63/63 [==============================] - 16s 253ms/step - loss: 0.5492 - accuracy: 0.8685 - val_loss: 0.1984 - val_accuracy: 0.9560
Epoch 4/30
63/63 [==============================] - 16s 256ms/step - loss: 0.3429 - accuracy: 0.9060 - val_loss: 0.1438 - val_accuracy: 0.9630
Epoch 5/30
63/63 [==============================] - 16s 250ms/step - loss: 0.2459 - accuracy: 0.9225 - val_loss: 0.1135 - val_accuracy: 0.9660
Epoch 6/30
63/63 [==============================] - 16s 253ms/step - loss: 0.1735 - accuracy: 0.9445 - val_loss: 0.1126 - val_accuracy: 0.9710
Epoch 7/30
63/63 [==============================] - 13s 194ms/step - loss: 0.1855 - accuracy: 0.9425 - val_loss: 0.1156 - val_accuracy: 0.9740

In [17]:
from tensorflow.keras.models import load_model

test_model = load_model(MODEL_PATH)
test_loss, test_acc = test_model.evaluate(test_data)

print(f"Test accuracy: {test_acc}")

63/63 [==============================] - 8s 111ms/step - loss: 0.1208 - accuracy: 0.9755
Test accuracy: 0.9754999876022339
